Retriver and Chain with Langchain

In [47]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("sample.pdf")
docs=loader.load()
docs

[Document(metadata={'producer': 'PyPDF', 'creator': 'PyPDF', 'creationdate': '2008-09-16T00:39:50+04:00', 'moddate': '2009-10-16T23:21:09+02:00', 'source': 'sample.pdf', 'total_pages': 462, 'page': 0, 'page_label': '1'}, page_content=''),
 Document(metadata={'producer': 'PyPDF', 'creator': 'PyPDF', 'creationdate': '2008-09-16T00:39:50+04:00', 'moddate': '2009-10-16T23:21:09+02:00', 'source': 'sample.pdf', 'total_pages': 462, 'page': 1, 'page_label': '2'}, page_content='Clean Code'),
 Document(metadata={'producer': 'PyPDF', 'creator': 'PyPDF', 'creationdate': '2008-09-16T00:39:50+04:00', 'moddate': '2009-10-16T23:21:09+02:00', 'source': 'sample.pdf', 'total_pages': 462, 'page': 2, 'page_label': '3'}, page_content='Robert C. Martin Series\nThe mission of this series is to improve the state of the art of software craftsmanship. \nThe books in this series are technical, pragmatic, and substantial. The authors are \nhighly experienced craftsmen and professionals dedicated to writing about w

In [48]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
documents = text_splitter.split_documents(docs)
documents[:5]

[Document(metadata={'producer': 'PyPDF', 'creator': 'PyPDF', 'creationdate': '2008-09-16T00:39:50+04:00', 'moddate': '2009-10-16T23:21:09+02:00', 'source': 'sample.pdf', 'total_pages': 462, 'page': 1, 'page_label': '2'}, page_content='Clean Code'),
 Document(metadata={'producer': 'PyPDF', 'creator': 'PyPDF', 'creationdate': '2008-09-16T00:39:50+04:00', 'moddate': '2009-10-16T23:21:09+02:00', 'source': 'sample.pdf', 'total_pages': 462, 'page': 2, 'page_label': '3'}, page_content='Robert C. Martin Series\nThe mission of this series is to improve the state of the art of software craftsmanship. \nThe books in this series are technical, pragmatic, and substantial. The authors are \nhighly experienced craftsmen and professionals dedicated to writing about what \nactually works in practice, as opposed to what might work in theory. You will read \nabout what the author has done, not what he thinks you should do. If the book is \nabout programming, there will be lots of code. If the book is abo

In [49]:
documents= text_splitter.split_documents(docs)
documents

[Document(metadata={'producer': 'PyPDF', 'creator': 'PyPDF', 'creationdate': '2008-09-16T00:39:50+04:00', 'moddate': '2009-10-16T23:21:09+02:00', 'source': 'sample.pdf', 'total_pages': 462, 'page': 1, 'page_label': '2'}, page_content='Clean Code'),
 Document(metadata={'producer': 'PyPDF', 'creator': 'PyPDF', 'creationdate': '2008-09-16T00:39:50+04:00', 'moddate': '2009-10-16T23:21:09+02:00', 'source': 'sample.pdf', 'total_pages': 462, 'page': 2, 'page_label': '3'}, page_content='Robert C. Martin Series\nThe mission of this series is to improve the state of the art of software craftsmanship. \nThe books in this series are technical, pragmatic, and substantial. The authors are \nhighly experienced craftsmen and professionals dedicated to writing about what \nactually works in practice, as opposed to what might work in theory. You will read \nabout what the author has done, not what he thinks you should do. If the book is \nabout programming, there will be lots of code. If the book is abo

In [50]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS 
db = FAISS.from_documents(documents[:20], HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2"))

In [51]:
db

In [52]:
query="What is the main topic of the document?"
result=db.similarity_search(query)
result[0].page_content

'Robert C. Martin Series\nThe mission of this series is to improve the state of the art of software craftsmanship. \nThe books in this series are technical, pragmatic, and substantial. The authors are \nhighly experienced craftsmen and professionals dedicated to writing about what \nactually works in practice, as opposed to what might work in theory. You will read \nabout what the author has done, not what he thinks you should do. If the book is \nabout programming, there will be lots of code. If the book is about managing, there \nwill be lots of case studies from real projects. \nThese are the books that all serious practitioners will have on their bookshelves. \nThese are the books that will be remembered for making a difference and for guiding \nprofessionals to become true craftsman. \nManaging Agile Projects\nSanjiv Augustine\nAgile Estimating and Planning\nMike Cohn\nWorking Effectively with Legacy Code\nMichael C. Feathers\nAgile Java™: Crafting Code with Test-Driven Developmen

In [53]:
from langchain_google_genai import ChatGoogleGenerativeAI
import os
from dotenv import load_dotenv

load_dotenv()

## Load Google Gemini LLM model
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0.7,
    google_api_key=os.getenv('GOOGLE_API_KEY')
)
llm

ChatGoogleGenerativeAI(model='models/gemini-1.5-flash', google_api_key=SecretStr('**********'), client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x000002339ED9E1D0>, default_metadata=(), model_kwargs={})

In [54]:
## Design Chatprompt Template
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template(""" 
Answer the following question based only on provided context.
Think step by step before providing a detailed answer.
 I will tip you $1000 if user finds answer helpful.
    <context>
    {context}
    </context>
    Question :{input}""")

In [55]:
## Create stuff document chain using LCEL (LangChain Expression Language)
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# Helper function to format documents
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Create the chain using LCEL
document_chain = prompt | llm | StrOutputParser()

# To use it with documents:
# response = document_chain.invoke({"context": format_docs(documents), "input": "your question"})

In [56]:
""" 
Retrievers: A retriever is an interface that returns documents given an unstructured query. It is more general than
a vector store,A retriever does not need to store documents,only to return(or retrieve) them.Vector stores can be as
backbone of retrievers,but there are other types of retrievers as well,such as those that use keyword search or web search.
 https://python.langchain.com/docs/modules/data_connection/retrievers/"""

retriever = db.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000233A02768D0>, search_kwargs={})

In [ ]:
"""Retriever Chain: This chain takes user inquiry, which is then passed to the retriever to fetch relevant documents.
These documents (and the original input) are then passed to an LLM to generate the final response.
https://python.langchain.com/docs/modules/chains/
"""
from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,document_chain)